**LOAD DATA**


In [ ]:
import pandas as pd

# Load the spreadsheet
file_path = '/content/online_retail_II.xlsx'
xls = pd.ExcelFile(file_path)

# Load the sheets into separate DataFrames
df_2009_2010 = pd.read_excel(xls, sheet_name='Year 2009-2010')
df_2010_2011 = pd.read_excel(xls, sheet_name='Year 2010-2011')

# Preview the first few rows to understand the structure
df_2009_2010.head(), df_2010_2011.head()


(  Invoice StockCode                          Description  Quantity  \
 0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
 1  489434    79323P                   PINK CHERRY LIGHTS        12   
 2  489434    79323W                  WHITE CHERRY LIGHTS        12   
 3  489434     22041         RECORD FRAME 7" SINGLE SIZE         48   
 4  489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   
 
           InvoiceDate  Price  Customer ID         Country  
 0 2009-12-01 07:45:00   6.95      13085.0  United Kingdom  
 1 2009-12-01 07:45:00   6.75      13085.0  United Kingdom  
 2 2009-12-01 07:45:00   6.75      13085.0  United Kingdom  
 3 2009-12-01 07:45:00   2.10      13085.0  United Kingdom  
 4 2009-12-01 07:45:00   1.25      13085.0  United Kingdom  ,
   Invoice StockCode                          Description  Quantity  \
 0  536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
 1  536365     71053                  WHITE METAL LANTERN      

**1.Checking Missing Value**

In [ ]:
# Check for missing values in both datasets
df_2009_2010.isnull().sum(), df_2010_2011.isnull().sum()


(Invoice             0
 StockCode           0
 Description      2928
 Quantity            0
 InvoiceDate         0
 Price               0
 Customer ID    107927
 Country             0
 dtype: int64,
 Invoice             0
 StockCode           0
 Description      1454
 Quantity            0
 InvoiceDate         0
 Price               0
 Customer ID    135080
 Country             0
 dtype: int64)

**Removing Data where CustomerID and Description are Null**

In [ ]:
# Drop rows where both 'Customer ID' and 'Description' are null
df_2009_2010_cleaned = df_2009_2010.dropna(subset=['Customer ID', 'Description'], how='all')
df_2010_2011_cleaned = df_2010_2011.dropna(subset=['Customer ID', 'Description'], how='all')

# Verify the cleaning by checking the number of remaining missing values in 'Customer ID' and 'Description'
df_2009_2010_cleaned[['Customer ID', 'Description']].isnull().sum(), df_2010_2011_cleaned[['Customer ID', 'Description']].isnull().sum()

(Customer ID    104999
 Description         0
 dtype: int64,
 Customer ID    133626
 Description         0
 dtype: int64)

**CHNAGE DATE FORMAT IN EXCEL**

In [ ]:
# Convert InvoiceDate to datetime
df_2009_2010['InvoiceDate'] = pd.to_datetime(df_2009_2010['InvoiceDate'], errors='coerce')
df_2010_2011['InvoiceDate'] = pd.to_datetime(df_2010_2011['InvoiceDate'], errors='coerce')

# Check if the conversion was successful
df_2009_2010['InvoiceDate'].head(), df_2010_2011['InvoiceDate'].head()


(0   2009-12-01 07:45:00
 1   2009-12-01 07:45:00
 2   2009-12-01 07:45:00
 3   2009-12-01 07:45:00
 4   2009-12-01 07:45:00
 Name: InvoiceDate, dtype: datetime64[ns],
 0   2010-12-01 08:26:00
 1   2010-12-01 08:26:00
 2   2010-12-01 08:26:00
 3   2010-12-01 08:26:00
 4   2010-12-01 08:26:00
 Name: InvoiceDate, dtype: datetime64[ns])

**Delete Row where Quantity value is Negative**

In [ ]:
# Filter rows where Quantity is negative (if any)
negative_quantity_2009_2010 = df_2009_2010[df_2009_2010['Quantity'] < 0]
negative_quantity_2010_2011 = df_2010_2011[df_2010_2011['Quantity'] < 0]

# Optionally, drop rows with negative Quantity or treat them as returns
df_2009_2010_cleaned = df_2009_2010[df_2009_2010['Quantity'] > 0]
df_2010_2011_cleaned = df_2010_2011[df_2010_2011['Quantity'] > 0]



Handle Duplicates

In [ ]:
# Check for duplicate rows based on Invoice and StockCode
duplicates_2009_2010 = df_2009_2010[df_2009_2010.duplicated(subset=['Invoice', 'StockCode','Description','Customer ID' ])]
duplicates_2010_2011 = df_2010_2011[df_2010_2011.duplicated(subset=['Invoice', 'StockCode','Description','Customer ID'])]

# Optionally, drop duplicates
df_2009_2010_cleaned = df_2009_2010.drop_duplicates(subset=['Invoice', 'StockCode','Description','Customer ID'])
df_2010_2011_cleaned = df_2010_2011.drop_duplicates(subset=['Invoice', 'StockCode','Description','Customer ID'])


In [ ]:
df_2009_2010_cleaned.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [ ]:
# Create a copy of the DataFrames to avoid modifying a view
df_2009_2010_cleaned = df_2009_2010_cleaned.copy()
df_2010_2011_cleaned = df_2010_2011_cleaned.copy()

# Remove non-numeric characters from 'StockCode' (keeping only numbers)
df_2009_2010_cleaned['StockCode'] = df_2009_2010_cleaned['StockCode'].astype(str).str.extract('(\d+)', expand=False)
df_2010_2011_cleaned['StockCode'] = df_2010_2011_cleaned['StockCode'].astype(str).str.extract('(\d+)', expand=False)

# Verify the changes
df_2009_2010_cleaned['StockCode'].head(), df_2010_2011_cleaned['StockCode'].head()


(0    85048
 1    79323
 2    79323
 3    22041
 4    21232
 Name: StockCode, dtype: object,
 0    85123
 1    71053
 2    84406
 3    84029
 4    84029
 Name: StockCode, dtype: object)

In [ ]:
# Standardize the 'Country' column (remove leading/trailing spaces and convert to title case)
df_2009_2010_cleaned['Country'] = df_2009_2010_cleaned['Country'].str.strip().str.title()
df_2010_2011_cleaned['Country'] = df_2010_2011_cleaned['Country'].str.strip().str.title()

# Check the unique values in 'Country' to ensure consistency
df_2009_2010_cleaned['Country'].unique(), df_2010_2011_cleaned['Country'].unique()


(array(['United Kingdom', 'France', 'Usa', 'Belgium', 'Australia', 'Eire',
        'Germany', 'Portugal', 'Japan', 'Denmark', 'Nigeria',
        'Netherlands', 'Poland', 'Spain', 'Channel Islands', 'Italy',
        'Cyprus', 'Greece', 'Norway', 'Austria', 'Sweden',
        'United Arab Emirates', 'Finland', 'Switzerland', 'Unspecified',
        'Malta', 'Bahrain', 'Rsa', 'Bermuda', 'Hong Kong', 'Singapore',
        'Thailand', 'Israel', 'Lithuania', 'West Indies', 'Lebanon',
        'Korea', 'Brazil', 'Canada', 'Iceland'], dtype=object),
 array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
        'Norway', 'Eire', 'Switzerland', 'Spain', 'Poland', 'Portugal',
        'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
        'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
        'Israel', 'Finland', 'Bahrain', 'Greece', 'Hong Kong', 'Singapore',
        'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
        'Czech Republic', 'Canada', 'Unspeci

In [ ]:
# Check for missing values in numeric columns (Quantity, Price)
df_2009_2010_cleaned[['Quantity', 'Price']].isnull().sum(), df_2010_2011_cleaned[['Quantity', 'Price']].isnull().sum()

# Optionally fill missing numeric values (e.g., with 0 or the median)
df_2009_2010_cleaned[['Quantity', 'Price']] = df_2009_2010_cleaned[['Quantity', 'Price']].fillna(0)
df_2010_2011_cleaned[['Quantity', 'Price']] = df_2010_2011_cleaned[['Quantity', 'Price']].fillna(0)


In [ ]:
# Remove rows where 'Invoice' contains non-numeric values
df_2009_2010_cleaned = df_2009_2010_cleaned[df_2009_2010_cleaned['Invoice'].astype(str).str.isnumeric()]
df_2010_2011_cleaned = df_2010_2011_cleaned[df_2010_2011_cleaned['Invoice'].astype(str).str.isnumeric()]

# Verify the changes by checking the first few rows
df_2009_2010_cleaned['Invoice'].head(), df_2010_2011_cleaned['Invoice'].head()


(0    489434
 1    489434
 2    489434
 3    489434
 4    489434
 Name: Invoice, dtype: object,
 0    536365
 1    536365
 2    536365
 3    536365
 4    536365
 Name: Invoice, dtype: object)

In [ ]:
# Save the cleaned data to new Excel files
df_2009_2010_cleaned.to_excel('cleaned_online_retail_2009_2010.xlsx', index=False)
df_2010_2011_cleaned.to_excel('cleaned_online_retail_2010_2011.xlsx', index=False)
